# 예측모델 전처리

In [1]:
# 모듈 Importing 및 경고 끄기
import warnings
warnings.filterwarnings(action='ignore')

import random
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

In [2]:
# 데이터 불러오기
df_data = pd.read_csv('./train_male.csv')
df_predict = pd.read_csv('./test_male.csv')
df_data.drop(['Unnamed: 0'], axis = 1, inplace = True)
df_predict.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [3]:
# 불러온 데이터 확인
df_data

,결과,득점_득점,공격종합성공률,후위성공률,퀵오픈성공률,서브성공률,디그성공률,세트성공률,리시브성공률,블로킹성공률,...,팀명_우리카드,팀명_한국전력,팀명_현대캐피탈,상대팀_KB손해보험,상대팀_OK금융그룹,상대팀_대한항공,상대팀_삼성화재,상대팀_우리카드,상대팀_한국전력,상대팀_현대캐피탈
0,1,76,0.525862,0.382353,0.714286,0.097087,0.804348,0.517857,0.481928,0.102041,...,0,0,0,0,0,0,1,0,0,0
1,1,50,0.523810,0.470588,0.500000,0.114286,0.883721,0.479452,0.454545,0.243243,...,0,0,0,0,0,0,0,0,0,1
2,0,59,0.438095,0.458333,0.517241,0.075000,0.809524,0.413793,0.493333,0.109375,...,0,0,0,0,0,0,0,0,1,0
3,1,48,0.480519,0.529412,0.733333,0.122449,0.729730,0.492958,0.340426,0.208333,...,0,0,0,0,0,1,0,0,0,0
4,0,81,0.538462,0.620690,0.468750,0.074074,0.800000,0.486239,0.510204,0.175439,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215,0,68,0.508929,0.458333,0.583333,0.000000,0.729167,0.472441,0.562500,0.177419,...,0,0,1,0,0,0,0,1,0,0
1216,0,25,0.478261,0.375000,0.500000,0.021277,0.740741,0.454545,0.425532,0.090909,...,0,0,1,0,0,0,1,0,0,0
1217,0,62,0.475248,0.500000,0.645161,0.035714,0.629630,0.448980,0.623188,0.229167,...,0,0,1,0,0,1,0,0,0,0
1218,0,35,0.476923,0.388889,0.647059,0.025641,0.805556,0.455696,0.383333,0.107143,...,0,0,1,1,0,0,0,0,0,0


In [4]:
# 각팀 별 데이터 DF 생성
KB = df_data[df_data['팀명_KB손해보험'] == 1]
KB = KB.drop(columns=['팀명_KB손해보험', '팀명_OK금융그룹', '팀명_대한항공', '팀명_삼성화재', '팀명_한국전력', '팀명_현대캐피탈','팀명_우리카드','상대팀_KB손해보험'])

OK = df_data[df_data['팀명_OK금융그룹'] == 1]
OK = OK.drop(columns=['팀명_KB손해보험', '팀명_OK금융그룹', '팀명_대한항공', '팀명_삼성화재', '팀명_한국전력', '팀명_현대캐피탈','팀명_우리카드','상대팀_OK금융그룹'])

대한 = df_data[df_data['팀명_대한항공'] == 1]
대한 = 대한.drop(columns=['팀명_KB손해보험', '팀명_OK금융그룹', '팀명_대한항공', '팀명_삼성화재', '팀명_한국전력', '팀명_현대캐피탈','팀명_우리카드','상대팀_대한항공'])

삼성 = df_data[df_data['팀명_삼성화재'] == 1]
삼성 = 삼성.drop(columns=['팀명_KB손해보험', '팀명_OK금융그룹', '팀명_대한항공', '팀명_삼성화재', '팀명_한국전력', '팀명_현대캐피탈','팀명_우리카드','상대팀_삼성화재'])

우리 = df_data[df_data['팀명_우리카드'] == 1]
우리 = 우리.drop(columns=['팀명_KB손해보험', '팀명_OK금융그룹', '팀명_대한항공', '팀명_삼성화재', '팀명_한국전력', '팀명_현대캐피탈','팀명_우리카드','상대팀_우리카드'])

한전 = df_data[df_data['팀명_한국전력'] == 1]
한전 = 한전.drop(columns=['팀명_KB손해보험', '팀명_OK금융그룹', '팀명_대한항공', '팀명_삼성화재', '팀명_한국전력', '팀명_현대캐피탈','팀명_우리카드','상대팀_한국전력'])

현대 = df_data[df_data['팀명_현대캐피탈'] == 1]
현대 = 현대.drop(columns=['팀명_KB손해보험', '팀명_OK금융그룹', '팀명_대한항공', '팀명_삼성화재', '팀명_한국전력', '팀명_현대캐피탈','팀명_우리카드','상대팀_현대캐피탈'])

In [5]:
# 각 팀별 예측대상 데이터 DF 생성
KB_predict = df_predict[df_predict['팀명_KB손해보험'] == 1]
KB_predict = KB_predict.drop(columns=['팀명_KB손해보험', '팀명_OK금융그룹', '팀명_대한항공', '팀명_삼성화재', '팀명_한국전력', '팀명_현대캐피탈','팀명_우리카드','상대팀_KB손해보험'])
KB_predict =KB_predict.reset_index().drop(columns='index')

OK_predict = df_predict[df_predict['팀명_OK금융그룹'] == 1]
OK_predict = OK_predict.drop(columns=['팀명_KB손해보험', '팀명_OK금융그룹', '팀명_대한항공', '팀명_삼성화재', '팀명_한국전력', '팀명_현대캐피탈','팀명_우리카드','상대팀_OK금융그룹'])
OK_predict =OK_predict.reset_index().drop(columns='index')

대한_predict = df_predict[df_predict['팀명_대한항공'] == 1]
대한_predict = 대한_predict.drop(columns=['팀명_KB손해보험', '팀명_OK금융그룹', '팀명_대한항공', '팀명_삼성화재', '팀명_한국전력', '팀명_현대캐피탈','팀명_우리카드','상대팀_대한항공'])
대한_predict =대한_predict.reset_index().drop(columns='index')

삼성_predict = df_predict[df_predict['팀명_삼성화재'] == 1]
삼성_predict = 삼성_predict.drop(columns=['팀명_KB손해보험', '팀명_OK금융그룹', '팀명_대한항공', '팀명_삼성화재', '팀명_한국전력', '팀명_현대캐피탈','팀명_우리카드','상대팀_삼성화재'])
삼성_predict =삼성_predict.reset_index().drop(columns='index')

우리_predict = df_predict[df_predict['팀명_우리카드'] == 1]
우리_predict = 우리_predict.drop(columns=['팀명_KB손해보험', '팀명_OK금융그룹', '팀명_대한항공', '팀명_삼성화재', '팀명_한국전력', '팀명_현대캐피탈','팀명_우리카드','상대팀_우리카드'])
우리_predict =우리_predict.reset_index().drop(columns='index')

한전_predict = df_predict[df_predict['팀명_한국전력'] == 1]
한전_predict = 한전_predict.drop(columns=['팀명_KB손해보험', '팀명_OK금융그룹', '팀명_대한항공', '팀명_삼성화재', '팀명_한국전력', '팀명_현대캐피탈','팀명_우리카드','상대팀_한국전력'])
한전_predict =한전_predict.reset_index().drop(columns='index')

현대_predict = df_predict[df_predict['팀명_현대캐피탈'] == 1]
현대_predict = 현대_predict.drop(columns=['팀명_KB손해보험', '팀명_OK금융그룹', '팀명_대한항공', '팀명_삼성화재', '팀명_한국전력', '팀명_현대캐피탈','팀명_우리카드','상대팀_현대캐피탈'])
현대_predict =현대_predict.reset_index().drop(columns='index')

# 각 팀 별 모델 만들기

팀별 학습 진행후, 제일 좋은 모델을 기반으로 팀, 상대팀 입력  
팀의 스탯은 평균값을 중심으로 min~max 범위내로 조정해서 함수 입력  
상대팀과의 상성과 임의로 주어진 스탯을 중심으로 승패 계산  

In [6]:
# 분류 모델 생성 및 하이퍼 파라미터 지정
lgb = LGBMClassifier(bagging_fraction=0.9, bagging_freq=3, boosting_type='gbdt',
                class_weight=None, feature_fraction=0.5,
                importance_type='split', learning_rate=0.4, max_depth=-1,
                min_child_samples=6, min_child_weight=0.001, min_split_gain=0.3,
                n_estimators=20, n_jobs=-1, num_leaves=150, objective=None,
                random_state=0, reg_alpha=0.005, reg_lambda=0.0005,
                silent='warn', subsample_for_bin=200000)
rf = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                        class_weight='balanced_subsample', criterion='entropy',
                        max_depth=4, max_features='log2', max_leaf_nodes=None,
                        max_samples=None, min_impurity_decrease=0.0002,
                        min_samples_leaf=5,
                        min_samples_split=9, min_weight_fraction_leaf=0.0,
                        n_estimators=130, n_jobs=-1, oob_score=False,
                        random_state=0, verbose=0, warm_start=False)
gbc = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                            learning_rate=0.01, loss='deviance', max_depth=7,
                            max_features='sqrt', max_leaf_nodes=None,
                            min_impurity_decrease=0.05,
                            min_samples_leaf=2, min_samples_split=2,
                            min_weight_fraction_leaf=0.0, n_estimators=140,
                            n_iter_no_change=None,
                            random_state=0, subsample=0.35, tol=0.0001,
                            validation_fraction=0.1, verbose=0,
                            warm_start=False)


In [7]:
# 각 팀별 모델 피팅
# KB손해보험 

model_KB = VotingClassifier([('lgb', lgb), ('rf',rf), ('gbc',gbc)], voting= 'soft')
X= KB.drop(columns=['결과'])
y = KB['결과']
model_KB.fit(X,y)

#OK금융그룹 

model_OK = VotingClassifier([('lgb', lgb), ('rf',rf), ('gbc',gbc)], voting= 'soft')
X= OK.drop(columns=['결과'])
y = OK['결과']
model_OK.fit(X,y)

#대한항공 

model_대한 = VotingClassifier([('lgb', lgb), ('rf',rf), ('gbc',gbc)], voting= 'soft')
X= 대한.drop(columns=['결과'])
y = 대한['결과']
model_대한.fit(X,y)

#삼성화재 

model_삼성 = VotingClassifier([('lgb', lgb), ('rf',rf), ('gbc',gbc)], voting= 'soft')
X= 삼성.drop(columns=['결과'])
y = 삼성['결과']
model_삼성.fit(X,y)

#우리카드 

model_우리 = VotingClassifier([('lgb', lgb), ('rf',rf), ('gbc',gbc)], voting= 'soft')
X= 우리.drop(columns=['결과'])
y = 우리['결과']
model_우리.fit(X,y)

#한국전력 

model_한전 = VotingClassifier([('lgb', lgb), ('rf',rf), ('gbc',gbc)], voting= 'soft')
X= 한전.drop(columns=['결과'])
y = 한전['결과']
model_한전.fit(X,y)

#현대캐피탈 

model_현대 = VotingClassifier([('lgb', lgb), ('rf',rf), ('gbc',gbc)], voting= 'soft')
X= 현대.drop(columns=['결과'])
y = 현대['결과']
model_현대.fit(X,y)

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3


VotingClassifier(estimators=[('lgb',
                              LGBMClassifier(bagging_fraction=0.9,
                                             bagging_freq=3,
                                             feature_fraction=0.5,
                                             learning_rate=0.4,
                                             min_child_samples=6,
                                             min_split_gain=0.3,
                                             n_estimators=20, num_leaves=150,
                                             random_state=0, reg_alpha=0.005,
                                             reg_lambda=0.0005)),
                             ('rf',
                              RandomForestClassifier(class_weight='balanced_subsample',
                                                     criterion='entropy',
                                                     max_depth=4,
                                                     max_features='log2',
             

In [8]:
# 랜덤 스탯 부여 함수
def pre_predict(x,x_predict):
    for i in range(len(x_predict)):
        if x_predict.iloc[i,10] == 1:
            x_predict.loc[i,'득점_득점'] = random.choice(x[x.iloc[:,10]==1]['득점_득점'].values)
            x_predict.loc[i,'공격종합성공률'] = random.choice(x[x.iloc[:,10]==1]['공격종합성공률'].values)
            x_predict.loc[i,'후위성공률'] = random.choice(x[x.iloc[:,10]==1]['후위성공률'].values)
            x_predict.loc[i,'퀵오픈성공률'] = random.choice(x[x.iloc[:,10]==1]['퀵오픈성공률'].values)
            x_predict.loc[i,'서브성공률'] = random.choice(x[x.iloc[:,10]==1]['서브성공률'].values)
            x_predict.loc[i,'디그성공률'] = random.choice(x[x.iloc[:,10]==1]['디그성공률'].values)
            x_predict.loc[i,'세트성공률'] = random.choice(x[x.iloc[:,10]==1]['세트성공률'].values)
            x_predict.loc[i,'리시브성공률'] = random.choice(x[x.iloc[:,10]==1]['리시브성공률'].values)
            x_predict.loc[i,'블로킹성공률'] = random.choice(x[x.iloc[:,10]==1]['블로킹성공률'].values)      
        if x_predict.iloc[i,11] == 1:
            x_predict.loc[i,'득점_득점'] = random.choice(x[x.iloc[:,11]==1]['득점_득점'].values)
            x_predict.loc[i,'공격종합성공률'] = random.choice(x[x.iloc[:,11]==1]['공격종합성공률'].values)
            x_predict.loc[i,'후위성공률'] = random.choice(x[x.iloc[:,11]==1]['후위성공률'].values)
            x_predict.loc[i,'퀵오픈성공률'] = random.choice(x[x.iloc[:,11]==1]['퀵오픈성공률'].values)
            x_predict.loc[i,'서브성공률'] = random.choice(x[x.iloc[:,11]==1]['서브성공률'].values)
            x_predict.loc[i,'디그성공률'] = random.choice(x[x.iloc[:,11]==1]['디그성공률'].values)
            x_predict.loc[i,'세트성공률'] = random.choice(x[x.iloc[:,11]==1]['세트성공률'].values)
            x_predict.loc[i,'리시브성공률'] = random.choice(x[x.iloc[:,11]==1]['리시브성공률'].values)
            x_predict.loc[i,'블로킹성공률'] = random.choice(x[x.iloc[:,11]==1]['블로킹성공률'].values)
        if x_predict.iloc[i,12] == 1: 
            x_predict.loc[i,'득점_득점'] = random.choice(x[x.iloc[:,12]==1]['득점_득점'].values)
            x_predict.loc[i,'공격종합성공률'] = random.choice(x[x.iloc[:,12]==1]['공격종합성공률'].values)
            x_predict.loc[i,'후위성공률'] = random.choice(x[x.iloc[:,12]==1]['후위성공률'].values)
            x_predict.loc[i,'퀵오픈성공률'] = random.choice(x[x.iloc[:,12]==1]['퀵오픈성공률'].values)
            x_predict.loc[i,'서브성공률'] = random.choice(x[x.iloc[:,12]==1]['서브성공률'].values)
            x_predict.loc[i,'디그성공률'] = random.choice(x[x.iloc[:,12]==1]['디그성공률'].values)
            x_predict.loc[i,'세트성공률'] = random.choice(x[x.iloc[:,12]==1]['세트성공률'].values)
            x_predict.loc[i,'리시브성공률'] = random.choice(x[x.iloc[:,12]==1]['리시브성공률'].values)
            x_predict.loc[i,'블로킹성공률'] = random.choice(x[x.iloc[:,12]==1]['블로킹성공률'].values)
        if x_predict.iloc[i,13] == 1:  
            x_predict.loc[i,'득점_득점'] = random.choice(x[x.iloc[:,13]==1]['득점_득점'].values)
            x_predict.loc[i,'공격종합성공률'] = random.choice(x[x.iloc[:,13]==1]['공격종합성공률'].values)
            x_predict.loc[i,'후위성공률'] = random.choice(x[x.iloc[:,13]==1]['후위성공률'].values)
            x_predict.loc[i,'퀵오픈성공률'] = random.choice(x[x.iloc[:,13]==1]['퀵오픈성공률'].values)
            x_predict.loc[i,'서브성공률'] = random.choice(x[x.iloc[:,13]==1]['서브성공률'].values)
            x_predict.loc[i,'디그성공률'] = random.choice(x[x.iloc[:,13]==1]['디그성공률'].values)
            x_predict.loc[i,'세트성공률'] = random.choice(x[x.iloc[:,13]==1]['세트성공률'].values)
            x_predict.loc[i,'리시브성공률'] = random.choice(x[x.iloc[:,13]==1]['리시브성공률'].values)
            x_predict.loc[i,'블로킹성공률'] = random.choice(x[x.iloc[:,13]==1]['블로킹성공률'].values)
        if x_predict.iloc[i,14] == 1:       
            x_predict.loc[i,'득점_득점'] = random.choice(x[x.iloc[:,14]==1]['득점_득점'].values)
            x_predict.loc[i,'공격종합성공률'] = random.choice(x[x.iloc[:,14]==1]['공격종합성공률'].values)
            x_predict.loc[i,'후위성공률'] = random.choice(x[x.iloc[:,14]==1]['후위성공률'].values)
            x_predict.loc[i,'퀵오픈성공률'] = random.choice(x[x.iloc[:,14]==1]['퀵오픈성공률'].values)
            x_predict.loc[i,'서브성공률'] = random.choice(x[x.iloc[:,14]==1]['서브성공률'].values)
            x_predict.loc[i,'디그성공률'] = random.choice(x[x.iloc[:,14]==1]['디그성공률'].values)
            x_predict.loc[i,'세트성공률'] = random.choice(x[x.iloc[:,14]==1]['세트성공률'].values)
            x_predict.loc[i,'리시브성공률'] = random.choice(x[x.iloc[:,14]==1]['리시브성공률'].values)
            x_predict.loc[i,'블로킹성공률'] = random.choice(x[x.iloc[:,14]==1]['블로킹성공률'].values)
        if x_predict.iloc[i,15] == 1:            
            x_predict.loc[i,'득점_득점'] = random.choice(x[x.iloc[:,15]==1]['득점_득점'].values)
            x_predict.loc[i,'공격종합성공률'] = random.choice(x[x.iloc[:,15]==1]['공격종합성공률'].values)
            x_predict.loc[i,'후위성공률'] = random.choice(x[x.iloc[:,15]==1]['후위성공률'].values)
            x_predict.loc[i,'퀵오픈성공률'] = random.choice(x[x.iloc[:,15]==1]['퀵오픈성공률'].values)
            x_predict.loc[i,'서브성공률'] = random.choice(x[x.iloc[:,15]==1]['서브성공률'].values)
            x_predict.loc[i,'디그성공률'] = random.choice(x[x.iloc[:,15]==1]['디그성공률'].values)
            x_predict.loc[i,'세트성공률'] = random.choice(x[x.iloc[:,15]==1]['세트성공률'].values)
            x_predict.loc[i,'리시브성공률'] = random.choice(x[x.iloc[:,15]==1]['리시브성공률'].values)
            x_predict.loc[i,'블로킹성공률'] = random.choice(x[x.iloc[:,15]==1]['블로킹성공률'].values)
            
    predict_data = pd.DataFrame(x_predict.drop(columns='결과'))
    return predict_data

In [9]:
# 승패 예측 함수
def winning_predict(model,x):
    win_lose = model.predict(x)
    for i in range(len(win_lose)):
        if win_lose[i] == 1:
            if x.iloc[i,9] == 1:
                print(f'{x.columns[9]}에게 승리')
            elif x.iloc[i,10] == 1:
                print(f'{x.columns[10]}에게 승리')            
            elif x.iloc[i,11] == 1:
                print(f'{x.columns[11]}에게 승리')
            elif x.iloc[i,12] == 1:
                print(f'{x.columns[12]}에게 승리')
            elif x.iloc[i,13] == 1:
                    print(f'{x.columns[13]}에게 승리')    
            elif x.iloc[i,14] == 1:
                    print(f'{x.columns[14]}에게 승리')    
        else:
            if x.iloc[i,9] == 1:
                print(f'{x.columns[9]}에게 패배')
            elif x.iloc[i,10] == 1:
                print(f'{x.columns[10]}에게 패배')            
            elif x.iloc[i,11] == 1:
                print(f'{x.columns[11]}에게 패배')
            elif x.iloc[i,12] == 1:
                print(f'{x.columns[12]}에게 패배')
            elif x.iloc[i,13] == 1:
                print(f'{x.columns[13]}에게 패배')    
            elif x.iloc[i,14] == 1:
                print(f'{x.columns[14]}에게 패배')       

In [14]:
# 입출력 구현
team_model_list =  [model_KB,model_OK,model_대한,model_삼성,model_우리,model_한전,model_현대]
team_name_list =  'model_KB,model_OK,model_대한,model_삼성,model_우리,model_한전,model_현대'.split(',')
predict_list = [KB_predict,OK_predict,대한_predict,삼성_predict,우리_predict,한전_predict,현대_predict]
team_list = [KB,OK,대한,삼성,우리,한전,현대]

while True:
    team_name = input(f'팀 이름을 입력하세요(KB, OK, 대한, 삼성, 우리, 한전, 현대): ')
    team_name_save = team_name
    team_name = 'model_'+team_name
    predict_name = team_name + '_predict'
    print(team_name)
    if team_name in team_name_list:
        for i in range(len(team_name_list)):
            if team_name == team_name_list[i]:
                team_name = team_model_list[i]
                predict_name = predict_list[i]
                team_name_save = team_list[i]
                break
        x = pre_predict(team_name_save,predict_name)
        winning_predict(team_name, x)
        pred = team_name.predict(x)      
        rate = round(len(predict_name[predict_name['결과'] == pred]) / len(predict_name)*100,2)
        print(f'적중률 : {rate}%')
        break
    else:
        continue

팀 이름을 입력하세요(KB, OK, 대한, 삼성, 우리, 한전, 현대): OK
model_OK
상대팀_우리카드에게 패배
상대팀_우리카드에게 승리
상대팀_한국전력에게 승리
상대팀_KB손해보험에게 패배
상대팀_삼성화재에게 패배
상대팀_대한항공에게 패배
상대팀_현대캐피탈에게 패배
적중률 : 42.86%
